In [4]:
import os
import glob
from osgeo import gdal

def process_subfolder(input_subfolder, output_subfolder, translate_options=None):
    os.makedirs(output_subfolder, exist_ok=True)
    output_mosaic_path = os.path.join(output_subfolder, "final_output.tif")

    # Check if the output mosaic file already exists
    if os.path.exists(output_mosaic_path):
        print(f"Output mosaic already exists for {input_subfolder}, skipping...")
        return

    input_files = sorted(glob.glob(os.path.join(input_subfolder, "*.tif")))

    if not input_files:
        print(f"No TIFF files found in {input_subfolder}")
        return

    print(f"Processing subfolder: {input_subfolder}")

    # Build VRT for the subfolder
    vrt_path = os.path.join(output_subfolder, "subfolder.vrt")
    gdal.BuildVRT(vrt_path, input_files)

    # Translate VRT to regular TIFF (not COG)
    gdal.Translate(output_mosaic_path, vrt_path, options=translate_options)

    # Clean up VRT
    os.remove(vrt_path)

    print(f"Final mosaic for {input_subfolder} created at: {output_mosaic_path}")

def create_mosaic_from_subfolders(input_root, output_root, test_mode=False):

    gdal.SetConfigOption('GDAL_CACHEMAX', '4096')
    gdal.SetConfigOption('GDAL_NUM_THREADS', 'ALL_CPUS')
    gdal.SetConfigOption('GDAL_MEMORY_LIMIT', '4096')
    # GDAL Translate Options - focus on speed, normal TIFF output
    translate_options = gdal.TranslateOptions(
        format='GTiff',  # Standard GeoTIFF format
        creationOptions=[
            "COMPRESS=LZW",  # Optional compression for speed, you can remove this if compression affects performance
            "BIGTIFF=YES",   # Enable support for large files
            "NUM_THREADS=ALL_CPUS"  # Utilize all available CPUs

        ]
    )

    # Walk through the input directory structure
    for root, dirs, files in os.walk(input_root):
        # Only process directories that contain TIFF files
        tiff_files = [f for f in files if f.lower().endswith('.tif')]
        if tiff_files:
            relative_path = os.path.relpath(root, input_root)
            input_subfolder = root
            output_subfolder = os.path.join(output_root, relative_path)

            # If test_mode is True, limit the number of files
            if test_mode:
                tiff_files = tiff_files[:1000]

            process_subfolder(input_subfolder, output_subfolder, translate_options)

    print("All subfolders processed.")



In [5]:
if __name__ == "__main__":
    # Replace these paths with your actual paths
    input_root = "/Volumes/Expansion 1/DTM_UNZIP"
    output_root = "/Volumes/Expansion 1/DTM_TIFF"
    test_mode = False  # Set to True to process a smaller subset for testing

    create_mosaic_from_subfolders(input_root, output_root, test_mode)


Processing subfolder: /Volumes/Expansion 1/DTM_UNZIP/Belleville-DTM-A


/Users/b/Documents/GitHub/ontario-dtm/.conda/lib/python3.12/site-packages/osgeo/gdal.py:312: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


Final mosaic for /Volumes/Expansion 1/DTM_UNZIP/Belleville-DTM-A created at: /Volumes/Expansion 1/DTM_TIFF/Belleville-DTM-A/final_output.tif
Processing subfolder: /Volumes/Expansion 1/DTM_UNZIP/Belleville-DTM-B
Final mosaic for /Volumes/Expansion 1/DTM_UNZIP/Belleville-DTM-B created at: /Volumes/Expansion 1/DTM_TIFF/Belleville-DTM-B/final_output.tif
Processing subfolder: /Volumes/Expansion 1/DTM_UNZIP/Belleville-DTM-C
Final mosaic for /Volumes/Expansion 1/DTM_UNZIP/Belleville-DTM-C created at: /Volumes/Expansion 1/DTM_TIFF/Belleville-DTM-C/final_output.tif
Processing subfolder: /Volumes/Expansion 1/DTM_UNZIP/Belleville-DTM-D
Final mosaic for /Volumes/Expansion 1/DTM_UNZIP/Belleville-DTM-D created at: /Volumes/Expansion 1/DTM_TIFF/Belleville-DTM-D/final_output.tif
Processing subfolder: /Volumes/Expansion 1/DTM_UNZIP/Belleville-DTM-E
Final mosaic for /Volumes/Expansion 1/DTM_UNZIP/Belleville-DTM-E created at: /Volumes/Expansion 1/DTM_TIFF/Belleville-DTM-E/final_output.tif
Processing subf

In [2]:
import os
import subprocess

# Define the root directory containing subfolders with raster tiles
input_dir = "/Volumes/Expansion 1/DTM_UNZIP"

# Function to execute shell commands
def run_gdal_command(cmd):
    result = subprocess.run(cmd, shell=True, capture_output=True, text=True)
    if result.returncode != 0:
        print(f"Error: {result.stderr}")
    else:
        print(f"Success: {result.stdout}")

# Process each subfolder in the input directory
for subfolder in os.listdir(input_dir):
    subfolder_path = os.path.join(input_dir, subfolder)
    
    if os.path.isdir(subfolder_path):
        output_mosaic = os.path.join(subfolder_path, "mosaic.tif")

        # Skip if output mosaic already exists
        if os.path.exists(output_mosaic):
            print(f"Skipping {subfolder}: mosaic.tif already exists.")
            continue
        
        print(f"Processing folder: {subfolder_path}")
        
        # Find all TIFF files in the subfolder
        tiff_files = [os.path.join(subfolder_path, f) for f in os.listdir(subfolder_path) if f.endswith(".tif")]

        if tiff_files:
            # Create VRT file
            vrt_file = os.path.join(subfolder_path, "mosaic.vrt")
            vrt_command = f"gdalbuildvrt {vrt_file} {' '.join(tiff_files)}"
            run_gdal_command(vrt_command)

            # Convert VRT to GeoTIFF
            translate_command = f"gdal_translate {vrt_file} {output_mosaic} -co COMPRESS=LZW"
            run_gdal_command(translate_command)

            # Remove VRT after conversion
            os.remove(vrt_file)
        else:
            print(f"No TIFF files found in {subfolder_path}")


Processing folder: /Volumes/Expansion 1/DTM_UNZIP/Belleville-DTM-A
Error: Warning 1: Can't open 1/DTM_UNZIP/Belleville-DTM-A/mosaic.vrt. Skipping it
Warning 1: Can't open /Volumes/Expansion. Skipping it
Warning 1: Can't open 1/DTM_UNZIP/Belleville-DTM-A/1km182830488802022LBELLEVILLE_DTM.tif. Skipping it
Warning 1: Can't open /Volumes/Expansion. Skipping it
Warning 1: Can't open 1/DTM_UNZIP/Belleville-DTM-A/1km182840488402022LBELLEVILLE_DTM.tif. Skipping it
Warning 1: Can't open /Volumes/Expansion. Skipping it
Warning 1: Can't open 1/DTM_UNZIP/Belleville-DTM-A/1km182840488502022LBELLEVILLE_DTM.tif. Skipping it
Warning 1: Can't open /Volumes/Expansion. Skipping it
Warning 1: Can't open 1/DTM_UNZIP/Belleville-DTM-A/1km182840488602022LBELLEVILLE_DTM.tif. Skipping it
Warning 1: Can't open /Volumes/Expansion. Skipping it
Warning 1: Can't open 1/DTM_UNZIP/Belleville-DTM-A/1km182840488702022LBELLEVILLE_DTM.tif. Skipping it
Warning 1: Can't open /Volumes/Expansion. Skipping it
Warning 1: Can't 

FileNotFoundError: [Errno 2] No such file or directory: '/Volumes/Expansion 1/DTM_UNZIP/Belleville-DTM-A/mosaic.vrt'